<a href="https://colab.research.google.com/github/TALeonard/19ma573thomasleonard/blob/master/src/hw7_es_bsm_knock_in_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, import modules and supply parameters.

In [0]:
import numpy as np
import scipy.stats as ss

In [0]:
'''=======
Parameters
======='''

T = 0.25
r = 0.05
sigma = 0.15
S0 = 95.0
K = 96.0
H = 85.0
num_step = 50
num_paths = 1000
num_trials = 1000 #this is small k, to prevent me from accidentally mistyping.

Now, define the Brownian Motion. Use same code as before.

In [0]:
def BM_gen(T1, T2, n): #para: start time, end time, mesh numbr
  t = np.linspace(T1, T2, num = n+1)
  W = np.zeros(n+1)
  #Run (3.2)
  for i in range(n):
    W[i+1] = W[i] + 1./np.sqrt(n) * np.random.normal()
    
  return t, W

Now define the Monte Carlo simulation code.

In [0]:
def knock_in_gbm_exact_sample(num_step,num_paths,s0,strike,r,sigma,maturity,H):
  
  m = num_step;
  n = num_paths;
  
  #Set up stock history array.
  stock_history = np.zeros((n,m+1))
  stock_history[:,0] = s0;
  
  #Set up array of payoffs.
  payoff = np.zeros(n)
  
  #Set up Indicator Vector for barrier option.
  min_indic = np.zeros(n)
  
  #Generate GBM paths and compute payoffs.
  
  for i in range(n):
    [t,W] = BM_gen(0,maturity,m)
    for j in range(m):
      stock_history[i,j+1] = s0 * np.exp((r-(0.5*(sigma**2)))*t[j+1] + sigma*W[j+1])
      if stock_history[i,j+1] < H:
        min_indic[i] = 1
      else:
        pass;
    if stock_history[i,m-1] - strike > 0:
      payoff[i] = 1000 * min_indic[i] #This will be 1000 if barrier is reached, 0 otherwise
    else:
      payoff[i] = 0
  
  return payoff.mean() * np.exp(-1*r*maturity)

Now, run k = 1000 trials (num_trials) and find the mean, MSE and CI.

In [12]:
#Create array to store means in.
payoff_means = np.zeros(num_trials)

for i in range(num_trials):
  payoff_means[i] = knock_in_gbm_exact_sample(num_step,num_paths,S0,K,r,sigma,T,H)

overall_mean = payoff_means.mean()

# Computing MSE: We require the "true" value of the price for this computation.
# Consider either using the average of averages (under the Law of Large Numbers)
# or computing the true price of such an option (the "continuous" price) directly.
# NEED TO ASK PROFESSOR

# Computing CI: Need the variance of the set. We can assume normality due to LLN.

overall_variance = payoff_means.var()
overall_sd = overall_variance ** 0.5

# No confidence interval is supplied. Will ask professor, but will use 95% CI 
# Until an answer is received.
# For 95% CI, the Z-score is 1.96

z_score = 1.96
interval_range = z_score * overall_sd / np.sqrt(num_trials)
lower_interval = overall_mean - interval_range
upper_interval = overall_mean + interval_range

Mean using 1000 MC outputs is 42.23178948251985
MSE using1000 MC outputs is 
Confidence interval using 1000 MC outputs is (41.842002555239624, 42.62157640980008)


In [13]:
# Printed outputs are in another code window so that it doesn't have to run the thousands of iterations the above requires in order to print properly.

print("Mean using " + str(num_trials) + " MC outputs is " + str(overall_mean))
print("MSE using " + str(num_trials) + " MC outputs is ")
print("Confidence interval using " + str(num_trials) + " MC outputs is (" + str(lower_interval) + ", " + str(upper_interval) + ")")

Mean using 1000 MC outputs is 42.23178948251985
MSE using 1000 MC outputs is 
Confidence interval using 1000 MC outputs is (41.842002555239624, 42.62157640980008)
